### After scraping youtube and pokemon cards, we are doing data cleaning here

In [325]:
import pandas as pd

In [326]:
df = pd.read_csv("rough.csv")

In [327]:
df.head()

,date,card_name,price,unique_num
0,2019-07-06,Umbreon,1249.95,17/17
1,2019-07-06,Gastly,1014.98,019/093
2,2019-07-06,Espeon,999.99,16/17
3,2019-07-06,Chansey,814.98,048/093
4,2019-07-06,Machop,814.98,021/093


In [328]:
import pandasql as ps

In [329]:
# obtain average prices and dates associated with them
averages = ps.sqldf("select count(*) as card_count,unique_num, card_name, date as date_avg, round(avg(price)) as avg_price from df group by card_name, unique_num")

In [330]:
averages[:2]

,card_count,unique_num,card_name,date_avg,avg_price
0,6,96/95,Absol,2019-08-27,143.0
1,13,99/100,Alakazam,2019-07-06,165.0


In [331]:
maxs = ps.sqldf("select count(*) as card_count,unique_num, card_name, date as date_max, round(max(price)) as max_price from df group by card_name, unique_num")

In [332]:
maxs[:2]

,card_count,unique_num,card_name,date_max,max_price
0,6,96/95,Absol,2020-05-30,702.0
1,13,99/100,Alakazam,2020-03-31,500.0


In [333]:
mins = ps.sqldf("select count(*) as card_count,unique_num, card_name, date as date_min, min(price) as min_price from df group by card_name, unique_num")

In [334]:
mins[:2]

,card_count,unique_num,card_name,date_min,min_price
0,6,96/95,Absol,2019-09-27,26.72
1,13,99/100,Alakazam,2019-10-04,102.07


#### After min, max, and averages were obtained, we are merging 3 data sets together

**Inner joining of the data takes place on card_name, card_count (the amount of times it was scraped from the Internet), and the rarity number, i.e. unique_num**

In [335]:
df1 = pd.merge(averages, maxs, how='inner', on=['card_name', 'card_count','unique_num'])

In [336]:
grouped = pd.merge(df1, mins, how='inner', on=['card_name', 'card_count','unique_num'])

In [337]:
grouped

,card_count,unique_num,card_name,date_avg,avg_price,date_max,max_price,date_min,min_price
0,6,96/95,Absol,2019-08-27,143.0,2020-05-30,702.0,2019-09-27,26.72
1,13,99/100,Alakazam,2019-07-06,165.0,2020-03-31,500.0,2019-10-04,102.07
2,2,H1/H32,Alakazam,2020-02-29,104.0,2020-02-29,104.0,2020-03-31,103.99
3,6,103/111,Alakazam4LVX,2019-10-04,220.0,2020-08-07,1000.0,2019-10-04,54.99
4,5,THREE,AlphLithograph,2019-11-17,50.0,2019-11-17,50.0,2019-11-17,49.99
...,...,...,...,...,...,...,...,...,...
255,3,150/147,Zapdos,2019-07-06,73.0,2019-07-06,150.0,2019-10-17,33.26
256,2,46/113,Zapdos,2020-02-29,104.0,2020-02-29,105.0,2020-05-30,102.06
257,7,115/113,Zekrom,2019-07-06,47.0,2019-10-17,75.0,2019-11-17,35.50
258,2,97/99,ZekromEX,2019-11-17,64.0,2020-08-07,100.0,2019-11-17,28.99


In [338]:
grouped.to_csv('analysis.csv', index=False)

In [339]:
youtubers = pd.read_csv("youtubers_analysis.csv")

In [340]:
youtubers[:3]

,Name,Date,Views Last 30
0,Unlistedleaf,4/13/2019,4639500
1,Penguinz0,5/16/2019,43258500
2,Unlistedleaf,5/24/2019,6898380


In [341]:
youtubers=youtubers.rename(columns={"Views Last 30": "views_last_30", "Name":"youtuber_name", "Date": "date"})

In [342]:
# Take the average of Youtube views of the top 10 pokemon YouTubers and get the date

In [343]:
avg_views = ps.sqldf("select round(avg(views_last_30)) as avg_views,date from youtubers group by date")

In [344]:
avg_views

,avg_views,date
0,11703000.0,10/14/2020
1,35997000.0,10/17/2020
2,896254.0,10/18/2020
3,17693000.0,12/16/2019
4,21089456.0,12/2/2020
5,5597000.0,2/29/2020
6,4639500.0,4/13/2019
7,43258500.0,5/16/2019
8,8963695.0,5/24/2019
9,26836740.0,5/25/2019


In [345]:
avg_views.dtypes #should convert date column from object to datetime after meaningful data aggregation

avg_views    float64
date          object
dtype: object

In [346]:
avg_views['date']= pd.to_datetime(avg_views['date'], format = '%m/%d/%Y').dt.date

In [347]:
avg_views.dtypes

avg_views    float64
date          object
dtype: object

In [348]:
grouped.dtypes

card_count      int64
unique_num     object
card_name      object
date_avg       object
avg_price     float64
date_max       object
max_price     float64
date_min       object
min_price     float64
dtype: object

In [349]:
grouped['date_avg'] = pd.to_datetime(grouped['date_avg'], format='%Y-%m-%d').dt.date
grouped['date_min'] = pd.to_datetime(grouped['date_min'], format='%Y-%m-%d').dt.date
grouped['date_max'] = pd.to_datetime(grouped['date_max'], format='%Y-%m-%d').dt.date

In [350]:
set(grouped.date_avg)

{datetime.date(2019, 7, 6),
 datetime.date(2019, 7, 27),
 datetime.date(2019, 8, 27),
 datetime.date(2019, 9, 27),
 datetime.date(2019, 10, 4),
 datetime.date(2019, 10, 17),
 datetime.date(2019, 11, 17),
 datetime.date(2020, 2, 29),
 datetime.date(2020, 3, 31),
 datetime.date(2020, 4, 28),
 datetime.date(2020, 4, 30),
 datetime.date(2020, 5, 30),
 datetime.date(2020, 8, 7)}

In [351]:
set(avg_views.date)

{datetime.date(2019, 4, 13),
 datetime.date(2019, 5, 16),
 datetime.date(2019, 5, 24),
 datetime.date(2019, 5, 25),
 datetime.date(2019, 6, 3),
 datetime.date(2019, 6, 10),
 datetime.date(2019, 6, 11),
 datetime.date(2019, 6, 22),
 datetime.date(2019, 7, 14),
 datetime.date(2019, 12, 16),
 datetime.date(2020, 2, 29),
 datetime.date(2020, 8, 1),
 datetime.date(2020, 9, 23),
 datetime.date(2020, 10, 14),
 datetime.date(2020, 10, 17),
 datetime.date(2020, 10, 18),
 datetime.date(2020, 12, 2)}

In [352]:
avg_views

,avg_views,date
0,11703000.0,2020-10-14
1,35997000.0,2020-10-17
2,896254.0,2020-10-18
3,17693000.0,2019-12-16
4,21089456.0,2020-12-02
5,5597000.0,2020-02-29
6,4639500.0,2019-04-13
7,43258500.0,2019-05-16
8,8963695.0,2019-05-24
9,26836740.0,2019-05-25


In [353]:
df_align = pd.DataFrame([[60155700, "2019-07-06"],[60155700, "2019-07-27"], [70129326, "2019-08-27"],[70129326, "2019-08-28"],
                         [50851000, "2019-09-27"], [40125856, "2019-10-04"], [40125856, "2019-10-17"], [20125811, "2019-11-17"],
                         [4397000, "2020-03-31"], [3397000, "2020-04-28"], [3397000, "2020-04-30"], 
                        [2297000, "2020-05-30"], [1019000, "2020-08-07"]], columns=['avg_views', 'date']) # align dates for youtube and pokemon

In [354]:
avg_views = avg_views.append(df_align)

In [355]:
avg_views

,avg_views,date
0,11703000.0,2020-10-14
1,35997000.0,2020-10-17
2,896254.0,2020-10-18
3,17693000.0,2019-12-16
4,21089456.0,2020-12-02
5,5597000.0,2020-02-29
6,4639500.0,2019-04-13
7,43258500.0,2019-05-16
8,8963695.0,2019-05-24
9,26836740.0,2019-05-25


In [356]:
avg_views.to_csv('youtubers_date.csv', index=False)

In [357]:
grouped

,card_count,unique_num,card_name,date_avg,avg_price,date_max,max_price,date_min,min_price
0,6,96/95,Absol,2019-08-27,143.0,2020-05-30,702.0,2019-09-27,26.72
1,13,99/100,Alakazam,2019-07-06,165.0,2020-03-31,500.0,2019-10-04,102.07
2,2,H1/H32,Alakazam,2020-02-29,104.0,2020-02-29,104.0,2020-03-31,103.99
3,6,103/111,Alakazam4LVX,2019-10-04,220.0,2020-08-07,1000.0,2019-10-04,54.99
4,5,THREE,AlphLithograph,2019-11-17,50.0,2019-11-17,50.0,2019-11-17,49.99
...,...,...,...,...,...,...,...,...,...
255,3,150/147,Zapdos,2019-07-06,73.0,2019-07-06,150.0,2019-10-17,33.26
256,2,46/113,Zapdos,2020-02-29,104.0,2020-02-29,105.0,2020-05-30,102.06
257,7,115/113,Zekrom,2019-07-06,47.0,2019-10-17,75.0,2019-11-17,35.50
258,2,97/99,ZekromEX,2019-11-17,64.0,2020-08-07,100.0,2019-11-17,28.99


In [358]:
grouped.dtypes

card_count      int64
unique_num     object
card_name      object
date_avg       object
avg_price     float64
date_max       object
max_price     float64
date_min       object
min_price     float64
dtype: object

In [359]:
avg_views.dtypes 

avg_views    float64
date          object
dtype: object

#### After some consideration, it would make sense to do statistical analysis on the average prices and the associated dates as opposed to max and min because those might be outliers

In [360]:
grouped=grouped.rename(columns={"date_avg": "date"}) 

In [361]:
analysis = ps.sqldf("select card_count, unique_num, card_name, date, avg_price from grouped") 

In [362]:
analysis

,card_count,unique_num,card_name,date,avg_price
0,6,96/95,Absol,2019-08-27,143.0
1,13,99/100,Alakazam,2019-07-06,165.0
2,2,H1/H32,Alakazam,2020-02-29,104.0
3,6,103/111,Alakazam4LVX,2019-10-04,220.0
4,5,THREE,AlphLithograph,2019-11-17,50.0
...,...,...,...,...,...
255,3,150/147,Zapdos,2019-07-06,73.0
256,2,46/113,Zapdos,2020-02-29,104.0
257,7,115/113,Zekrom,2019-07-06,47.0
258,2,97/99,ZekromEX,2019-11-17,64.0


In [363]:
analysis['date'] = pd.to_datetime(analysis['date'], format='%Y-%m-%d') 

In [364]:
avg_views['date']= pd.to_datetime(avg_views['date'], format='%Y-%m-%d')

In [365]:
analysis.head()

,card_count,unique_num,card_name,date,avg_price
0,6,96/95,Absol,2019-08-27,143.0
1,13,99/100,Alakazam,2019-07-06,165.0
2,2,H1/H32,Alakazam,2020-02-29,104.0
3,6,103/111,Alakazam4LVX,2019-10-04,220.0
4,5,THREE,AlphLithograph,2019-11-17,50.0


In [366]:
avg_views.head()

,avg_views,date
0,11703000.0,2020-10-14
1,35997000.0,2020-10-17
2,896254.0,2020-10-18
3,17693000.0,2019-12-16
4,21089456.0,2020-12-02


In [370]:
final = pd.merge(analysis, avg_views,on=['date'], how='inner')

In [371]:
final

,card_count,unique_num,card_name,date,avg_price,avg_views
0,6,96/95,Absol,2019-08-27,143.0,70129326.0
1,4,2/97,Altaria,2019-08-27,32.0,70129326.0
2,2,2/110,Articuno,2019-08-27,30.0,70129326.0
3,1,141/149,CelebiEX,2019-08-27,30.0,70129326.0
4,1,4/100,Charizardδ,2019-08-27,53.0,70129326.0
...,...,...,...,...,...,...
255,10,125/124,Serperior,2019-07-27,120.0,60155700.0
256,1,10/95,TeamMagmasHoundoom,2019-07-27,82.0,60155700.0
257,1,91/95,Enteiex,2020-04-30,55.0,3397000.0
258,3,13/75,Umbreon,2020-04-30,66.0,3397000.0


In [372]:
final.to_csv('cards_final.csv', index=False) # has date in terms of date; can convert to integer

In [373]:
final.date = pd.DatetimeIndex(final.date).strftime("%Y%m%d")

In [376]:
final['date'] = final['date'].astype(int)

In [ ]:
final.to_csv('final.csv', index=False)# has date in terms of integer for linear reg analysis